In [ ]:
import json
from pprint import pprint
from copy import deepcopy

from tap_ebay.client import eBayClient

In [ ]:
import datetime
from ebaysdk.response import ResponseDataObject

from genson import SchemaBuilder
from genson.schema.strategies import TypedSchemaStrategy
        
class CustomDateTime(TypedSchemaStrategy):
    """
    strategy for date-time formatted strings
    """
    JS_TYPE = 'string'
    PYTHON_TYPE = (str, type(u''))

    # create a new instance variable
    def __init__(self, node_class):
        super().__init__(node_class)
        self.format = "date-time"

    @classmethod
    def match_object(self, obj):
        super().match_object(obj)
        try:
            if isinstance(obj, datetime.datetime):
                return True
            else:
                return False
        except (TypeError, ValueError) as exception:
            #print(exception)
            return False

    def to_schema(self):
        schema = super().to_schema()
        schema['type'] = self.JS_TYPE
        schema['format'] = self.format
        return schema

class CustomSchemaBuilder(SchemaBuilder):
    """ detects & labels date-time formatted strings """
    EXTRA_STRATEGIES = (CustomDateTime, )

In [ ]:
config = json.load(open(".secrets/config.json", "r"))

In [ ]:
client = eBayClient(app_id=config["app_id"])

In [ ]:
api = client.get_finding_api()

In [ ]:
verb = config["searches"][0]["verb"]
data = config["searches"][0]["data"]
response = client.execute(api, verb, data)

In [ ]:
dir(response.reply.paginationOutput)

In [ ]:
def get_all_items(client, api, verb, data, max_pages=100):
    items = []
    params = deepcopy(data)
    params["paginationInput"] =  {"entriesPerPage": 100}
    response = client.execute(api, verb, params)
    items.extend(response.reply.searchResult.item)
    page_number = int(response.reply.paginationOutput.pageNumber)
    total_pages = int(response.reply.paginationOutput.totalPages)
    while (page_number < total_pages) and (page_number < max_pages):
        page_number += 1
        params["paginationInput"] =  {"entriesPerPage": 100, "pageNumber": page_number}
        response = client.execute(api, verb, params)
        items.extend(response.reply.searchResult.item)
    return items

In [ ]:
items = get_all_items(client, api, verb, data, max_pages=10)

In [ ]:
def to_dict(obj):
    if isinstance(obj, ResponseDataObject):
        obj = obj.__dict__
        for key, value in obj.items():
            obj[key] = to_dict(value)
    return obj

In [ ]:
records = [to_dict(i) for i in items]

In [ ]:
len(records)

In [ ]:
int(response.reply.paginationOutput.totalEntries)

In [ ]:
builder = CustomSchemaBuilder()
for record in records:
    builder.add_object(record)
json_schema = builder.to_json(indent=2)
print(json_schema)

In [ ]:
with open("tap_ebay/schema/finding_item.json", "w") as search_item_schema:
    search_item_schema.write(json_schema)